In [1]:
from cmt.project import Project
from pathlib import Path
import json

ModuleNotFoundError: No module named 'cmt'

# Aanlevering vanuit de stochastentool

De CMT kan een export vanuit de Stochastentool inlezen. Deze levering bevat:
* een of meerdere modellen (in dit voorbeeld `dellen_rr_fm_rtc_T1.zip`)
* initiele condities (in dit voorbeeld `hydrosummerhalfyear_middelnat.zip` en `hydrowinterhalfyear_middelnat.zip`)
* eventueel randvoorwaarden
* een json met daarin de informatie hoe dit te vertalen naar cases `populate_cases.json`

Kijk maar!

In [ ]:
from_stochastentool = Path(r"../data/dellen")
[i.name for i in from_stochastentool.glob("*")]

Wat zit er ín populate_cases.json? Laten we het openen!

In [ ]:
populate_cases_json = from_stochastentool / "populate_cases.json"
populate_cases = json.loads(populate_cases_json.read_text())

print(f"De inhoud van {populate_cases_json.name} bevat:")
print(f" Aantal meteorologische randvoorwaarden: {len(populate_cases['boundary_conditions']['meteo'])}")
print(f" Aantal hydraulische randvoorwaarden: {len(populate_cases['boundary_conditions']['flow'])}")
print(f" Aantal initiele condities voor RR: {len(populate_cases['initial_conditions']['rr'])}")
print(f" Aantal modellen {len(populate_cases['initial_conditions'])}")
print(f" Aantal cases hieruit afgeleid {len(populate_cases['cases'])}")


We bekijken nu waarmee de eerste case moet worden opgebouwd en worden gedraaid

In [ ]:
case = populate_cases["cases"][0]
print(f"id van de case: {case['id']}")
print(f"de meteo die moet worden gebruikt: {case['meteo_bc_id']}")
print(f"de hydraulische rvw die moet worden gebruikt: {case['flow_bc_id']}")
print(f"de initiele conditie voor RR: {case['rr_ini_id']}")

De modulde `stowabui` is als onderdeel van deze TKI ontwikkeld voor het genereren van STOWA buien via Python. Deze module wordt hieronder geimporteerd. De MeteoEvent klasse binnen deze module maken we een standaard meteorologische forcing aan met een duur van 24 uur een bui-volume van 15mm het STOWA patroon `HOOG` en een verdamping van 0mm/dag. 

We maken hieronder de klasse aan en verifieren wat we hierboven beweren met een plotje. We printen ook de eigenschappen waarmee we een bui kunnen configureren:
- `meteo_event.duration`: de duur van de bui in een pd.Timedelta
- `meteo_event.volume.rainfall`: het volume van de neerslag over de duur
- `meteo_event.pattern`: het gebruikte buipatroon
- `meteo_event.season`: het gebruikte seizoen

Voor onze case zien deze parameters er als volgt uit

In [ ]:
meteo = next(i for i in populate_cases["boundary_conditions"]["meteo"] if i["id"] == case['meteo_bc_id'])

print(f"duur: {meteo['stowa_bui']['duration']} uur")
print(f"volume: {meteo['stowa_bui']['volume']} mm")
print(f"patroon: {meteo['stowa_bui']['pattern']}")
print(f"seizoen: {meteo['stowa_bui']['season']}")

# Aanmaken van cases

Vanuit de aangeleverde data uit de stochastentool bouwen we cases

In [ ]:
project = Project(filepath=r"../data/stochast").from_stochastics(populate_cases_json)

Hier alle cases die we hebben aangemaakt:

In [ ]:
[i.name for i in project.cases]

We vinden hier de vertaalde case uit de populate_cases.json terug:

In [ ]:
project.get_case(case["id"]).dict()

# Het runnen van 1 case
We draaien de eerste case met `stream_output=True`. Wat normaal in de DHYDRO console wordt gelogd, zie je nu in de notebook

In [ ]:
project.run_case(case["id"], stream_output=True)

Wanneer we nogmaals de case opvragn, zien we dat de run heeft gedraaid, wanneer de run is gestart en hoelang de run erover heeft gedaan.

In [ ]:
project.get_case(case["id"]).dict()

Een samenvatting van de resultaten voor een select aantal locaties wordt weggeschreven:

In [ ]:
project.results[0].results[case["id"]]

# Runnen in batches

Nu runnen we alles parallel:
* workers=3: we verdelen de last over 3 CPU's
* delete_output=True: we delete the fm output dir en slaan alleen de data op die we nodig hebben

In [ ]:
case_ids = project.get_cases()
project.run_cases(case_ids, workers=3, delete_output=True)